In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class LSTMPackedModelWithScalars(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size_seq, output_size_scalars):
        super(LSTMPackedModelWithScalars, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer for sequence output
        self.fc_sequence = nn.Linear(hidden_size, output_size_seq)

        # Fully connected layer for scalar output
        self.fc_scalar = nn.Linear(hidden_size, output_size_scalar)

    def forward(self, x, lengths):
        # Pack the padded sequence
        packed_input = rnn_utils.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)

        # LSTM forward pass
        packed_output, (hn, cn) = self.lstm(packed_input)

        # Unpack the sequence
        output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)
        #output = packed_output.data
        # Sequence prediction (for each time step)
        seq_output = self.fc_sequence(output)

        # Scalar prediction (based on the last hidden state of LSTM)
        # `hn` contains the hidden state for the last time step
        scalar_output = self.fc_scalar(hn[-1])  # Take the hidden state of the last LSTM layer

        return seq_output, scalar_output

# Example usage:
batch_size = 3
seq_lengths = torch.tensor([4, 2, 1])  # Variable lengths of the sequences
input_size = 2
hidden_size = 16
num_layers = 2
output_size_seq = 1  # Sequence output size
output_size_scalar = 2  # Scalars output size

model = LSTMPackedModelWithScalars(input_size, hidden_size, num_layers, output_size_seq, output_size_scalar)